In [ ]:
import sys

sys.path.append("../")
sys.path.append("../src/")

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

from src.utils import load_moons, load_monk1, load_mnist
from src.network import Network
from src.activations import ReLU, Tanh, Sigmoid
from src.losses import MeanSquaredError
from src.metrics import BinaryAccuracy, MulticlassAccuracy, MeanEuclideanError
from src.regularizers import L2, L1
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from src.callbacks import EarlyStopping, Callback

# MONK 1

In [ ]:
x_train, x_test, y_train, y_test = load_monk1(validation=False)

In [ ]:
model = Network(17)
model.add_layer(4, ReLU())
model.add_layer(1, Sigmoid())

In [ ]:
try:
    stats = model.train(
        (x_train, y_train),
        (x_test, y_test),
        metric=BinaryAccuracy(),
        loss=MeanSquaredError(),
        epochs=500,
        eta=0.01,
        nesterov=0.8,
        verbose=True,
    )
except KeyboardInterrupt:
    model.bar.close()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats["train_loss"], ax=axs[0])
sns.lineplot(stats["val_loss"], ax=axs[0])
sns.lineplot(stats["train_acc"], ax=axs[1])
sns.lineplot(stats["val_acc"], ax=axs[1])

## MONK 2

In [ ]:
from src.utils import load_monk2

x_train, x_test, y_train, y_test = load_monk2(validation=False)

model = Network(17)
model.add_layer(4, ReLU())
model.add_layer(1, Sigmoid(), initializer="xavier")

try:
    stats = model.train(
        (x_train, y_train),
        (x_test, y_test),
        metric=BinaryAccuracy(),
        loss=MeanSquaredError(),
        epochs=200,
        eta=0.01,
        nesterov=0.8,
        verbose=True,
    )
except KeyboardInterrupt:
    model.bar.close()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats["train_loss"], ax=axs[0])
sns.lineplot(stats["val_loss"], ax=axs[0])
sns.lineplot(stats["train_acc"], ax=axs[1])
sns.lineplot(stats["val_acc"], ax=axs[1])

## MONK Test 3

#### Trying without regularizer

In [ ]:
from src.utils import load_monk3

x_train, x_test, y_train, y_test = load_monk3(validation=False)

model = Network(17)
model.add_layer(4, ReLU())
model.add_layer(1, Sigmoid(), initializer="xavier")

try:
    stats = model.train(
        (x_train, y_train),
        (x_test, y_test),
        metric=BinaryAccuracy(),
        loss=MeanSquaredError(),
        epochs=200,
        eta=0.01,
        nesterov=0.8,
        verbose=True,
    )
except KeyboardInterrupt:
    model.bar.close()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats["train_loss"], ax=axs[0])
sns.lineplot(stats["val_loss"], ax=axs[0])
sns.lineplot(stats["train_acc"], ax=axs[1])
sns.lineplot(stats["val_acc"], ax=axs[1])

#### Monk's 3 without regularization started overfitting soon. Try regularizing with earlystopping.

In [ ]:
model = Network(17)
model.add_layer(4, ReLU())
model.add_layer(1, Sigmoid(), initializer="xavier")

try:
    stats = model.train(
        (x_train, y_train),
        (x_test, y_test),
        metric=BinaryAccuracy(),
        loss=MeanSquaredError(),
        epochs=500,
        eta=0.01,
        nesterov=0.8,
        callbacks=[EarlyStopping(patience=30)],
        verbose=True,
    )
except KeyboardInterrupt:
    model.bar.close()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats["train_loss"], ax=axs[0])
sns.lineplot(stats["val_loss"], ax=axs[0])
sns.lineplot(stats["train_acc"], ax=axs[1])
sns.lineplot(stats["val_acc"], ax=axs[1])

In [ ]:
# compute accuracy on test set
y_pred = model.multiple_outputs(x_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print("Accuracy on test set: ", accuracy_score(y_test.flatten(), y_pred.flatten()))

#### Much better, now try to regularize with L2 regularization.

In [ ]:
from src.regularizers import L2, L1

model = Network(17, L2(1e-5))
model.add_layer(4, ReLU())
model.add_layer(1, Sigmoid(), initializer="xavier")

try:
    stats = model.train(
        (x_train, y_train),
        (x_test, y_test),
        metric=BinaryAccuracy(),
        loss=MeanSquaredError(),
        epochs=500,
        eta=0.01,
        nesterov=0.8,
        callbacks=[EarlyStopping(patience=30)],
        verbose=True,
    )
except KeyboardInterrupt:
    model.bar.close()

In [ ]:
# compute accuracy on test set
y_pred = model.multiple_outputs(x_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print("Accuracy on test set: ", accuracy_score(y_test.flatten(), y_pred.flatten()))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.lineplot(stats["train_loss"], ax=axs[0])
sns.lineplot(stats["val_loss"], ax=axs[0])
sns.lineplot(stats["train_acc"], ax=axs[1])
sns.lineplot(stats["val_acc"], ax=axs[1])